# Scraping Concerts - Lab

## Introduction

Now that you've seen how to scrape a simple website, it's time to again practice those skills on a full-fledged site!
In this lab, you'll practice your scraping skills on a music website: https://www.residentadvisor.net.
## Objectives

You will be able to:
* Create a full scraping pipeline that involves traversing over many pages of a website, dealing with errors and storing data

## View the Website

For this lab, you'll be scraping the https://www.residentadvisor.net website. Start by navigating to the events page [here](https://www.residentadvisor.net/events) in your browser.

<img src="images/ra.png">

In [1]:
# Load the https://www.residentadvisor.net/events page in your browser.

## Open the Inspect Element Feature

Next, open the inspect element feature from your web browser in order to preview the underlying HTML associated with the page.

In [2]:
# Open the inspect element feature in your browser
import re
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

In [3]:
response = requests.get("https://www.residentadvisor.net/events/us/newyork")
soup = BeautifulSoup(response.content, 'html.parser')

In [4]:
print(soup.p)

<p>Fri, 13 Dec 2019</p>


In [5]:
soup.prettify

<bound method Tag.prettify of <!DOCTYPE html>

<html lang="en,ja,es">
<head id="_x1"><title>
	RA: Events in New York, United States of America
</title><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="en,ja,es" http-equiv="content-language"/><meta content="RA: Resident Advisor" name="Description"/><meta content="RA, residentadvisor, resident, advisor, music, ra, events, in, new, york, united, states, america" name="Keywords"/><meta content="Resident Advisor" name="Author"/><meta content="Resident Advisor" property="og:site_name"/><meta content="712773712080127" property="fb:app_id"/><link href="/bundles/default-css?v=FkfRVAlFvpndxqgZliJaJOXD-OhkiRFP8nrBK9Pg2R01" rel="stylesheet"/>
<meta content="app-id=981952703, app-argument=ra-guide://search" name="apple-itunes-app"/><link href="/bundles/cat-listings-css?v=qgpSmyPbylOKeJFqy2yvCrTgAsw9yQYcJtLKS_vPO6s1" rel="stylesheet"/>
<link href="/favicon.ico" rel="icon" type="image/vnd.microsoft.icon"/><link color=

In [6]:
event_listings = soup.find('div', id="event-listing")

In [7]:
display(len(event_listings))

3

In [8]:
entries = event_listings.findAll('li')
print(len(entries))

145


In [9]:
entries[0]

<li><p class="eventDate date"><a href="/events.aspx?ai=8&amp;v=day&amp;mn=12&amp;yr=2019&amp;dy=12"><span>Thu, 12 Dec 2019 /</span></a></p></li>

In [10]:
event = event_listings.findAll('h1')


In [11]:
event[0]

<h1 class="event-title" itemprop="summary"><a href="/events/1317360" itemprop="url" title="Event details of Guy J (All Night)">Guy J (All Night)</a> <span>at <a href="/club.aspx?id=160440">Lost Circus at Avant Gardner</a></span></h1>

In [12]:
rows = []
for entry in entries:
    #Is it a date? If so, set current date.
    date = entry.find('p', class_="eventDate date")
    event = entry.find('h1', class_="event-title")
    if event:
        details = event.text.split(' at ')
        event_name = details[0].strip()
        venue = details[1].strip()
        try:
            n_attendees = int(re.match("(\d*)", entry.find('p', class_="attending").text)[0])
        except:
            n_attendees = np.nan
        rows.append([event_name, venue, cur_date, n_attendees])
    elif date:
        cur_date = date.text
    else:
        continue
df = pd.DataFrame(rows)
df.head()
        
    

,0,1,2,3
0,Guy J (All Night),Lost Circus,"Thu, 12 Dec 2019 /",121.0
1,"Tourist x Matthew Dear Live - Brooklyn, NY",Avant Gardner,"Thu, 12 Dec 2019 /",20.0
2,"The Funky Seshwa with Rahaan, Rich Medina and ...",Good Room,"Thu, 12 Dec 2019 /",19.0
3,[CANCELLED] Memento Records Takeover,Electric Room,"Thu, 12 Dec 2019 /",5.0
4,Tourist x Matthew Dear,Avant Gardner,"Thu, 12 Dec 2019 /",19.0


## Write a Function to Scrape all of the Events on the Given Page Events Page

The function should return a Pandas DataFrame with columns for the Event_Name, Venue, Event_Date and Number_of_Attendees.

In [13]:
def scrape_events(events_page_url):
    response = requests.get(events_page_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    entries = event_listings.findAll('li')
    
    rows = []
    for entry in entries:
        date = entry.find('p', class_="eventDate date")
        event = entry.find('h1', class_="event-title")
        if event:
            details = event.text.split(' at ')
            event_name = details[0].strip()
            venue = details[1].strip()
            try:
                nu_attendees = int(re.match("(\d*)", entry.find('p', class_="attending").text)[0])
            except:
                nu_attendees = np.nan
        elif date:
            cur_date = date.text
        continue
            
    df = pd.DataFrame(rows)
    df.head()
    df.columns = ["Event_Name", "Venue", "Event_Date", "Number_of_Attendees"]
    return df  
df.head()


,0,1,2,3
0,Guy J (All Night),Lost Circus,"Thu, 12 Dec 2019 /",121.0
1,"Tourist x Matthew Dear Live - Brooklyn, NY",Avant Gardner,"Thu, 12 Dec 2019 /",20.0
2,"The Funky Seshwa with Rahaan, Rich Medina and ...",Good Room,"Thu, 12 Dec 2019 /",19.0
3,[CANCELLED] Memento Records Takeover,Electric Room,"Thu, 12 Dec 2019 /",5.0
4,Tourist x Matthew Dear,Avant Gardner,"Thu, 12 Dec 2019 /",19.0


## Write a Function to Retrieve the URL for the Next Page

In [14]:
soup.prettify

<bound method Tag.prettify of <!DOCTYPE html>

<html lang="en,ja,es">
<head id="_x1"><title>
	RA: Events in New York, United States of America
</title><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="en,ja,es" http-equiv="content-language"/><meta content="RA: Resident Advisor" name="Description"/><meta content="RA, residentadvisor, resident, advisor, music, ra, events, in, new, york, united, states, america" name="Keywords"/><meta content="Resident Advisor" name="Author"/><meta content="Resident Advisor" property="og:site_name"/><meta content="712773712080127" property="fb:app_id"/><link href="/bundles/default-css?v=FkfRVAlFvpndxqgZliJaJOXD-OhkiRFP8nrBK9Pg2R01" rel="stylesheet"/>
<meta content="app-id=981952703, app-argument=ra-guide://search" name="apple-itunes-app"/><link href="/bundles/cat-listings-css?v=qgpSmyPbylOKeJFqy2yvCrTgAsw9yQYcJtLKS_vPO6s1" rel="stylesheet"/>
<link href="/favicon.ico" rel="icon" type="image/vnd.microsoft.icon"/><link color=

In [19]:
soup.find('a', attrs={'ga-event-action':"Next "}).attrs['href']

'/events/us/newyork/week/2019-12-19'

In [20]:
def next_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    url_ext = soup.find('a', attrs={'ga-event-action':"Next "}).attrs['href']
    next_page_url = "https://www.residentadvisor.net" + url_ext
    #Your code here
    return next_page_url
  

## Scrape the Next 1000 Events for Your Area

Display the data sorted by the number of attendees. If there is a tie for the number attending, sort by event date.

In [21]:
dfs = []
total_rows = 0
cur_url = "https://www.residentadvisor.net/events/us/newyork"
while total_rows <= 1000:
    df = scrape_events(cur_url)
    dfs.append(df)
    total_rows += len(df)
    cur_url = next_page(cur_url)
    time.sleep(.2)
df = pd.concat(dfs)
df = df.iloc[:1000]
print(len(df))
df.head()

ValueError: Length mismatch: Expected axis has 0 elements, new values have 4 elements

## Summary 

Congratulations! In this lab, you successfully developed a pipeline to scrape a website for concert event information!